## LangGraph Open Deep Research - Supervisor-Researcher Architecture

In this notebook, we'll explore the **supervisor-researcher delegation architecture** for conducting deep research with LangGraph.

You can visit this repository to see the original application: [Open Deep Research](https://github.com/langchain-ai/open_deep_research)

Let's jump in!

## What We're Building

This implementation uses a **hierarchical delegation pattern** where:

1. **User Clarification** - Optionally asks clarifying questions to understand the research scope
2. **Research Brief Generation** - Transforms user messages into a structured research brief
3. **Supervisor** - A lead researcher that analyzes the brief and delegates research tasks
4. **Parallel Researchers** - Multiple sub-agents that conduct focused research simultaneously
5. **Research Compression** - Each researcher synthesizes their findings
6. **Final Report** - All findings are combined into a comprehensive report

![Architecture Diagram](https://i.imgur.com/Q8HEZn0.png)

This differs from a section-based approach by allowing dynamic task decomposition based on the research question, rather than predefined sections.

---

# 🤝 Breakout Room #1
## Deep Research Foundations

In this breakout room, we'll understand the architecture and components of the Open Deep Research system.

## Task 1: Dependencies

You'll need API keys for Anthropic (for the LLM) and Tavily (for web search). We'll configure the system to use Anthropic's Claude Sonnet 4 exclusively.

In [1]:
import os
import getpass

os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key: ")
os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

## Task 2: State Definitions

The state structure is hierarchical with three levels:

### Agent State (Top Level)
Contains the overall conversation messages, research brief, accumulated notes, and final report.

### Supervisor State (Middle Level)
Manages the research supervisor's messages, research iterations, and coordinating parallel researchers.

### Researcher State (Bottom Level)
Each individual researcher has their own message history, tool call iterations, and research findings.

We also have structured outputs for tool calling:
- **ConductResearch** - Tool for supervisor to delegate research to a sub-agent
- **ResearchComplete** - Tool to signal research phase is done
- **ClarifyWithUser** - Structured output for asking clarifying questions
- **ResearchQuestion** - Structured output for the research brief

Let's import these from our library: [`open_deep_library/state.py`](open_deep_library/state.py)

In [2]:
# Import state definitions from the library
from open_deep_library.state import (
    # Main workflow states
    AgentState,           # Lines 65-72: Top-level agent state with messages, research_brief, notes, final_report
    AgentInputState,      # Lines 62-63: Input state is just messages
    
    # Supervisor states
    SupervisorState,      # Lines 74-81: Supervisor manages research delegation and iterations
    
    # Researcher states
    ResearcherState,      # Lines 83-90: Individual researcher with messages and tool iterations
    ResearcherOutputState, # Lines 92-96: Output from researcher (compressed research + raw notes)
    
    # Structured outputs for tool calling
    ConductResearch,      # Lines 15-19: Tool for delegating research to sub-agents
    ResearchComplete,     # Lines 21-22: Tool to signal research completion
    ClarifyWithUser,      # Lines 30-41: Structured output for user clarification
    ResearchQuestion,     # Lines 43-48: Structured output for research brief
)

## Task 3: Utility Functions and Tools

The system uses several key utilities:

### Search Tools
- **tavily_search** - Async web search with automatic summarization to stay within token limits
- Supports Anthropic native web search and Tavily API

### Reflection Tools
- **think_tool** - Allows researchers to reflect on their progress and plan next steps (ReAct pattern)

### Helper Utilities
- **get_all_tools** - Assembles the complete toolkit (search + MCP + reflection)
- **get_today_str** - Provides current date context for research
- Token limit handling utilities for graceful degradation

These are defined in [`open_deep_library/utils.py`](open_deep_library/utils.py)

In [3]:
# Import utility functions and tools from the library
from open_deep_library.utils import (
    # Search tool - Lines 43-136: Tavily search with automatic summarization
    tavily_search,
    
    # Reflection tool - Lines 219-244: Strategic thinking tool for ReAct pattern
    think_tool,
    
    # Tool assembly - Lines 569-597: Get all configured tools
    get_all_tools,
    
    # Date utility - Lines 872-879: Get formatted current date
    get_today_str,
    
    # Supporting utilities for error handling
    get_api_key_for_model,          # Lines 892-914: Get API keys from config or env
    is_token_limit_exceeded,         # Lines 665-701: Detect token limit errors
    get_model_token_limit,           # Lines 831-846: Look up model's token limit
    remove_up_to_last_ai_message,    # Lines 848-866: Truncate messages for retry
    anthropic_websearch_called,      # Lines 607-637: Detect Anthropic native search usage
    openai_websearch_called,         # Lines 639-658: Detect OpenAI native search usage
    get_notes_from_tool_calls,       # Lines 599-601: Extract notes from tool messages
)

## Task 4: Configuration System

The configuration system controls:

### Research Behavior
- **allow_clarification** - Whether to ask clarifying questions before research
- **max_concurrent_research_units** - How many parallel researchers can run (default: 5)
- **max_researcher_iterations** - How many times supervisor can delegate research (default: 6)
- **max_react_tool_calls** - Tool call limit per researcher (default: 10)

### Model Configuration
- **research_model** - Model for research and supervision (we'll use Anthropic)
- **compression_model** - Model for synthesizing findings
- **final_report_model** - Model for writing the final report
- **summarization_model** - Model for summarizing web search results

### Search Configuration
- **search_api** - Which search API to use (ANTHROPIC, TAVILY, or NONE)
- **max_content_length** - Character limit before summarization

Defined in [`open_deep_library/configuration.py`](open_deep_library/configuration.py)

In [4]:
# Import configuration from the library
from open_deep_library.configuration import (
    Configuration,    # Lines 38-247: Main configuration class with all settings
    SearchAPI,        # Lines 11-17: Enum for search API options (ANTHROPIC, TAVILY, NONE)
)

## Task 5: Prompt Templates

The system uses carefully engineered prompts for each phase:

### Phase 1: Clarification
**clarify_with_user_instructions** - Analyzes if the research scope is clear or needs clarification

### Phase 2: Research Brief
**transform_messages_into_research_topic_prompt** - Converts user messages into a detailed research brief

### Phase 3: Supervisor
**lead_researcher_prompt** - System prompt for the supervisor that manages delegation strategy

### Phase 4: Researcher
**research_system_prompt** - System prompt for individual researchers conducting focused research

### Phase 5: Compression
**compress_research_system_prompt** - Prompt for synthesizing research findings without losing information

### Phase 6: Final Report
**final_report_generation_prompt** - Comprehensive prompt for writing the final report

All prompts are defined in [`open_deep_library/prompts.py`](open_deep_library/prompts.py)

In [5]:
# Import prompt templates from the library
from open_deep_library.prompts import (
    clarify_with_user_instructions,                    # Lines 3-41: Ask clarifying questions
    transform_messages_into_research_topic_prompt,     # Lines 44-77: Generate research brief
    lead_researcher_prompt,                            # Lines 79-136: Supervisor system prompt
    research_system_prompt,                            # Lines 138-183: Researcher system prompt
    compress_research_system_prompt,                   # Lines 186-222: Research compression prompt
    final_report_generation_prompt,                    # Lines 228-308: Final report generation
)

## ❓ Question #1:

Explain the interrelationships between the three states (Agent, Supervisor, Researcher). Why don't we just make a single huge state?

##### Answer:
Having separate states helps to manage and isolate context and improve task execution. It's been shown that LLMs experience context bloat - if a lot of information is added to the context, the models get confused and their performance worsens. Introducing hierarchy of states means one agent doesn't have to maintain all the context but can focus on specific sub-topics ,e.g. the subagent researcher can focus on doing deep research while the supervisor can focus on coordinating research work. Some tasks can be also done in parallel such as sub research, which is more efficient. The states are interrelated because selected data flows down through the agent hierarchy in a structured manner.



## ❓ Question #2:

What are the advantages and disadvantages of importing these components instead of including them in the notebook?

##### Answer:
In general, notebooks are used for exploratory work and learning since they are simpler to use. In production and when working on projects you use a folder structure with separate files so that the code is broken down into separate components. It's easier to maintain and more scalable.

Here, the advantage of importing components into the notebook makes it cleaner and higher level since the  implementation details are in separate files. The disadvantage of that is that if you don't look into those files you don't really understand how the splution is implemented; you also need to find the correct files, it's not laid out in a linear manner as it would in a notebook. Still this is more scalable from the project perspective and it's a common practice.



## 🏗️ Activity #1: Explore the Prompts

Open `open_deep_library/prompts.py` and examine one of the prompt templates in detail.

**Requirements:**
1. Choose one prompt template (clarify, brief, supervisor, researcher, compression, or final report)
2. Explain what the prompt is designed to accomplish
3. Identify 2-3 key techniques used in the prompt (e.g., structured output, role definition, examples)
4. Suggest one improvement you might make to the prompt

**YOUR CODE HERE** - Write your analysis in a markdown cell below

1. Supervisor prompt
   
2. The prompt sets out the role, the overall research task, lists available tools and provides key instructions.
   
3. The prompt leverages the following:
   - Clear role definition, task and workflow overview (sets out the steps in a linear fashion).
   - Structured format - indicates key subsections with markdown format, e.g. <Task> </Task>, and uses numbering to list guidelines.
   - Flexibility in use case-specific details by injecting additional info with placeholders, e.g. {messages}, {date}. this makes it reusable and scalable.
   - ICL by providing examples 

4. The prompt is decent overall but it is quite wordy and some info is repeated (feels it was vibe-coded with an LLM not curated by a human). I consolidated some of the instructions and rearranged the order a bit. I also added a note on "gathering results" - it was not very explicit that the ultimate point of research is to gather findings. I'm not expecting tremendous improvements but it reads better to a human. I would then test it and compare the performance to see if the change had any effect. 
   - renamed "instructions" to "steps" (everyhing is an instruction in the end and this shows more of a linear order).
   - consolidated "show your thinking" with "steps".
   - moved the line about "thinking like research manager" up to the beginning where the role is defined (it's high level).
   - added a note to "gather research findings" at the start of the prompt and at the end of the "Steps" section.


In [ ]:
# revised supervisor prompt
lead_researcher_prompt = """You are a research supervisor. Your job is to conduct research by calling the "ConductResearch" tool and gather findings. Think like a research manager with limited time and resources. For context, today's date is {date}.

<Task>
Your focus is to call the "ConductResearch" tool to conduct research against the overall research question passed in by the user. 
When you are completely satisfied with the research findings returned from the tool calls, then you should call the "ResearchComplete" tool to indicate that you are done with your research.
</Task>

<Available Tools>
You have access to three main tools:
1. **ConductResearch**: Delegate research tasks to specialized sub-agents
2. **ResearchComplete**: Indicate that research is complete
3. **think_tool**: For reflection and strategic planning during research. Use before and after each ConductResearch tool call to show your thinking.

**CRITICAL: Use think_tool before calling ConductResearch to plan your approach, and after each ConductResearch to assess progress. Do not call think_tool with any other tools in parallel.**
</Available Tools>

<Steps>
Follow these steps:
1. **Read the question carefully** - What specific information does the user need?
2. **Decide how to delegate the research** - Carefully considering the question, decide how to delegate the research. Are there multiple independent directions that can be explored simultaneously? Can the task be broken down into smaller sub-tasks? Use the think_tool to plan your approach.
4. **Call ConductResearch** - Call the ConductResearch tool to delegate the research task to a specialized sub-agent.
5. **After each call to ConductResearch, pause and assess** - What key information did I find? Do I have enough to answer? What's still missing? Should I delegate more research or call ResearchComplete? Do I have enough to answer the question comprehensively? Use the think_tool to analyze the results. 
6. **Call ResearchComplete** - When you are completely satisfied with the research findings returned from the tool calls, then you should call the "ResearchComplete" tool to indicate that you are done with gathering the research findings.
</Steps>

<Hard Limits>
**Task Delegation Budgets** (Prevent excessive delegation):
- **Bias towards single agent** - Use single agent for simplicity unless the user request has clear opportunity for parallelization
- **Stop when you can answer confidently** - Don't keep delegating research for perfection
- **Limit tool calls** - Always stop after {max_researcher_iterations} tool calls to ConductResearch and think_tool if you cannot find the right sources

**Maximum {max_concurrent_research_units} parallel agents per iteration**
</Hard Limits>

<Scaling Rules>
**Simple fact-finding, lists, and rankings** can use a single sub-agent:
- *Example*: List the top 10 coffee shops in San Francisco → Use 1 sub-agent

**Comparisons presented in the user request** can use a sub-agent for each element of the comparison:
- *Example*: Compare OpenAI vs. Anthropic vs. DeepMind approaches to AI safety → Use 3 sub-agents
- Delegate clear, distinct, non-overlapping subtopics

**Important Reminders:**
- Each ConductResearch call spawns a dedicated research agent for that specific topic
- A separate agent will write the final report - you just need to gather information
- When calling ConductResearch, provide complete standalone instructions - sub-agents can't see other agents' work
- Do NOT use acronyms or abbreviations in your research questions, be very clear and specific
</Scaling Rules>"""

---

# 🤝 Breakout Room #2
## Building & Running the Researcher

In this breakout room, we'll explore the node functions, build the graph, and run wellness research.

## Task 6: Node Functions - The Building Blocks

Now let's look at the node functions that make up our graph. We'll import them from the library and understand what each does.

### The Complete Research Workflow

The workflow consists of 8 key nodes organized into 3 subgraphs:

1. **Main Graph Nodes:**
   - `clarify_with_user` - Entry point that checks if clarification is needed
   - `write_research_brief` - Transforms user input into structured research brief
   - `final_report_generation` - Synthesizes all research into final report

2. **Supervisor Subgraph Nodes:**
   - `supervisor` - Lead researcher that plans and delegates
   - `supervisor_tools` - Executes supervisor's tool calls (delegation, reflection)

3. **Researcher Subgraph Nodes:**
   - `researcher` - Individual researcher conducting focused research
   - `researcher_tools` - Executes researcher's tool calls (search, reflection)
   - `compress_research` - Synthesizes researcher's findings

All nodes are defined in [`open_deep_library/deep_researcher.py`](open_deep_library/deep_researcher.py)

### Node 1: clarify_with_user

**Purpose:** Analyzes user messages and asks clarifying questions if the research scope is unclear.

**Key Steps:**
1. Check if clarification is enabled in configuration
2. Use structured output to analyze if clarification is needed
3. If needed, end with a clarifying question for the user
4. If not needed, proceed to research brief with verification message

**Implementation:** [`open_deep_library/deep_researcher.py` lines 60-115](open_deep_library/deep_researcher.py#L60-L115)

In [6]:
# Import the clarify_with_user node
from open_deep_library.deep_researcher import clarify_with_user

### Node 2: write_research_brief

**Purpose:** Transforms user messages into a structured research brief for the supervisor.

**Key Steps:**
1. Use structured output to generate detailed research brief from messages
2. Initialize supervisor with system prompt and research brief
3. Set up supervisor messages with proper context

**Why this matters:** A well-structured research brief helps the supervisor make better delegation decisions.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 118-175](open_deep_library/deep_researcher.py#L118-L175)

In [7]:
# Import the write_research_brief node
from open_deep_library.deep_researcher import write_research_brief

### Node 3: supervisor

**Purpose:** Lead research supervisor that plans research strategy and delegates to sub-researchers.

**Key Steps:**
1. Configure model with three tools:
   - `ConductResearch` - Delegate research to a sub-agent
   - `ResearchComplete` - Signal that research is done
   - `think_tool` - Strategic reflection before decisions
2. Generate response based on current context
3. Increment research iteration count
4. Proceed to tool execution

**Decision Making:** The supervisor uses `think_tool` to reflect before delegating research, ensuring thoughtful decomposition of the research question.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 178-223](open_deep_library/deep_researcher.py#L178-L223)

In [8]:
# Import the supervisor node (from supervisor subgraph)
from open_deep_library.deep_researcher import supervisor

### Node 4: supervisor_tools

**Purpose:** Executes the supervisor's tool calls, including strategic thinking and research delegation.

**Key Steps:**
1. Check exit conditions:
   - Exceeded maximum iterations
   - No tool calls made
   - `ResearchComplete` called
2. Process `think_tool` calls for strategic reflection
3. Execute `ConductResearch` calls in parallel:
   - Spawn researcher subgraphs for each delegation
   - Limit to `max_concurrent_research_units` (default: 5)
   - Gather all results asynchronously
4. Aggregate findings and return to supervisor

**Parallel Execution:** This is where the magic happens - multiple researchers work simultaneously on different aspects of the research question.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 225-349](open_deep_library/deep_researcher.py#L225-L349)

In [9]:
# Import the supervisor_tools node
from open_deep_library.deep_researcher import supervisor_tools

### Node 5: researcher

**Purpose:** Individual researcher that conducts focused research on a specific topic.

**Key Steps:**
1. Load all available tools (search, MCP, reflection)
2. Configure model with tools and researcher system prompt
3. Generate response with tool calls
4. Increment tool call iteration count

**ReAct Pattern:** Researchers use `think_tool` to reflect after each search, deciding whether to continue or provide their answer.

**Available Tools:**
- Search tools (Tavily or Anthropic native search)
- `think_tool` for strategic reflection
- `ResearchComplete` to signal completion
- MCP tools (if configured)

**Implementation:** [`open_deep_library/deep_researcher.py` lines 365-424](open_deep_library/deep_researcher.py#L365-L424)

In [10]:
# Import the researcher node (from researcher subgraph)
from open_deep_library.deep_researcher import researcher

### Node 6: researcher_tools

**Purpose:** Executes the researcher's tool calls, including searches and strategic reflection.

**Key Steps:**
1. Check early exit conditions (no tool calls, native search used)
2. Execute all tool calls in parallel:
   - Search tools fetch and summarize web content
   - `think_tool` records strategic reflections
   - MCP tools execute external integrations
3. Check late exit conditions:
   - Exceeded `max_react_tool_calls` (default: 10)
   - `ResearchComplete` called
4. Continue research loop or proceed to compression

**Error Handling:** Safely handles tool execution errors and continues with available results.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 435-509](open_deep_library/deep_researcher.py#L435-L509)

In [11]:
# Import the researcher_tools node
from open_deep_library.deep_researcher import researcher_tools

### Node 7: compress_research

**Purpose:** Compresses and synthesizes research findings into a concise, structured summary.

**Key Steps:**
1. Configure compression model
2. Add compression instruction to messages
3. Attempt compression with retry logic:
   - If token limit exceeded, remove older messages
   - Retry up to 3 times
4. Extract raw notes from tool and AI messages
5. Return compressed research and raw notes

**Why Compression?** Researchers may accumulate lots of tool outputs and reflections. Compression ensures:
- All important information is preserved
- Redundant information is deduplicated
- Content stays within token limits for the final report

**Token Limit Handling:** Gracefully handles token limit errors by progressively truncating messages.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 511-585](open_deep_library/deep_researcher.py#L511-L585)

In [12]:
# Import the compress_research node
from open_deep_library.deep_researcher import compress_research

### Node 8: final_report_generation

**Purpose:** Generates the final comprehensive research report from all collected findings.

**Key Steps:**
1. Extract all notes from completed research
2. Configure final report model
3. Attempt report generation with retry logic:
   - If token limit exceeded, truncate findings by 10%
   - Retry up to 3 times
4. Return final report or error message

**Token Limit Strategy:**
- First retry: Use model's token limit × 4 as character limit
- Subsequent retries: Reduce by 10% each time
- Graceful degradation with helpful error messages

**Report Quality:** The prompt guides the model to create well-structured reports with:
- Proper headings and sections
- Inline citations
- Comprehensive coverage of all findings
- Sources section at the end

**Implementation:** [`open_deep_library/deep_researcher.py` lines 607-697](open_deep_library/deep_researcher.py#L607-L697)

In [13]:
# Import the final_report_generation node
from open_deep_library.deep_researcher import final_report_generation

## Task 7: Graph Construction - Putting It All Together

The system is organized into three interconnected graphs:

### 1. Researcher Subgraph (Bottom Level)
Handles individual focused research on a specific topic:
```
START → researcher → researcher_tools → compress_research → END
               ↑            ↓
               └────────────┘ (loops until max iterations or ResearchComplete)
```

### 2. Supervisor Subgraph (Middle Level)
Manages research delegation and coordination:
```
START → supervisor → supervisor_tools → END
            ↑              ↓
            └──────────────┘ (loops until max iterations or ResearchComplete)
            
supervisor_tools spawns multiple researcher_subgraphs in parallel
```

### 3. Main Deep Researcher Graph (Top Level)
Orchestrates the complete research workflow:
```
START → clarify_with_user → write_research_brief → research_supervisor → final_report_generation → END
                 ↓                                       (supervisor_subgraph)
               (may end early if clarification needed)
```

Let's import the compiled graphs from the library.

In [14]:
# Import the pre-compiled graphs from the library
from open_deep_library.deep_researcher import (
    # Bottom level: Individual researcher workflow
    researcher_subgraph,    # Lines 588-605: researcher → researcher_tools → compress_research
    
    # Middle level: Supervisor coordination
    supervisor_subgraph,    # Lines 351-363: supervisor → supervisor_tools (spawns researchers)
    
    # Top level: Complete research workflow
    deep_researcher,        # Lines 699-719: Main graph with all phases
)

## Why This Architecture?

### Advantages of Supervisor-Researcher Delegation

1. **Dynamic Task Decomposition**
   - Unlike section-based approaches with predefined structure, the supervisor can break down research based on the actual question
   - Adapts to different types of research (comparisons, lists, deep dives, etc.)

2. **Parallel Execution**
   - Multiple researchers work simultaneously on different aspects
   - Much faster than sequential section processing
   - Configurable parallelism (1-20 concurrent researchers)

3. **ReAct Pattern for Quality**
   - Researchers use `think_tool` to reflect after each search
   - Prevents excessive searching and improves search quality
   - Natural stopping conditions based on information sufficiency

4. **Flexible Tool Integration**
   - Easy to add MCP tools for specialized research
   - Supports multiple search APIs (Anthropic, Tavily)
   - Each researcher can use different tool combinations

5. **Graceful Token Limit Handling**
   - Compression prevents token overflow
   - Progressive truncation in final report generation
   - Research can scale to arbitrary depths

### Trade-offs

- **Complexity:** More moving parts than section-based approach
- **Cost:** Parallel researchers use more tokens (but faster)
- **Unpredictability:** Research structure emerges dynamically

## Task 8: Running the Deep Researcher

Now let's see the system in action! We'll use it to research wellness strategies for improving sleep quality.

### Setup

We need to:
1. Set up the wellness research request
2. Configure the execution with Anthropic settings
3. Run the research workflow

In [15]:
# Set up the graph with Anthropic configuration
from IPython.display import Markdown, display
import uuid

# Note: deep_researcher is already compiled from the library
# For this demo, we'll use it directly without additional checkpointing
graph = deep_researcher

print("✓ Graph ready for execution")
print("  (Note: The graph is pre-compiled from the library)")

✓ Graph ready for execution
  (Note: The graph is pre-compiled from the library)


### Configuration for Anthropic

We'll configure the system to use:
- **Claude Sonnet 4** for all research, supervision, and report generation
- **Tavily** for web search (you can also use Anthropic's native search)
- **Moderate parallelism** (1 concurrent researcher for cost control)
- **Clarification enabled** (will ask if research scope is unclear)

In [16]:
# Configure for Anthropic with moderate settings
config = {
    "configurable": {
        # Model configuration - using Claude Sonnet 4 for everything
        "research_model": "anthropic:claude-sonnet-4-20250514",
        "research_model_max_tokens": 10000,
        
        "compression_model": "anthropic:claude-sonnet-4-20250514",
        "compression_model_max_tokens": 8192,
        
        "final_report_model": "anthropic:claude-sonnet-4-20250514",
        "final_report_model_max_tokens": 10000,
        
        "summarization_model": "anthropic:claude-sonnet-4-20250514",
        "summarization_model_max_tokens": 8192,
        
        # Research behavior
        "allow_clarification": True,
        "max_concurrent_research_units": 1,  # 1 parallel researcher
        "max_researcher_iterations": 2,      # Supervisor can delegate up to 2 times
        "max_react_tool_calls": 3,           # Each researcher can make up to 3 tool calls
        
        # Search configuration
        "search_api": "tavily",  # Using Tavily for web search
        "max_content_length": 50000,
        
        # Thread ID for this conversation
        "thread_id": str(uuid.uuid4())
    }
}

print("✓ Configuration ready")
print(f"  - Research Model: Claude Sonnet 4")
print(f"  - Max Concurrent Researchers: 1")
print(f"  - Max Iterations: 2")
print(f"  - Search API: Tavily")

✓ Configuration ready
  - Research Model: Claude Sonnet 4
  - Max Concurrent Researchers: 1
  - Max Iterations: 2
  - Search API: Tavily


### Execute the Wellness Research

Now let's run the research! We'll ask the system to research evidence-based strategies for improving sleep quality.

The workflow will:
1. **Clarify** - Check if the request is clear (may skip if obvious)
2. **Research Brief** - Transform our request into a structured brief
3. **Supervisor** - Plan research strategy and delegate to researchers
4. **Parallel Research** - Researchers gather information simultaneously
5. **Compression** - Each researcher synthesizes their findings
6. **Final Report** - All findings combined into comprehensive report

In [17]:
# Create our wellness research request
research_request = """
I want to improve my sleep quality. I currently:
- Go to bed at inconsistent times (10pm-1am)
- Use my phone in bed
- Often feel tired in the morning

Please research the best evidence-based strategies for improving sleep quality and create a comprehensive sleep improvement plan for me.
"""

# Execute the graph
async def run_research():
    """Run the research workflow and display results."""
    print("Starting research workflow...\n")
    
    async for event in graph.astream(
        {"messages": [{"role": "user", "content": research_request}]},
        config,
        stream_mode="updates"
    ):
        # Display each step
        for node_name, node_output in event.items():
            print(f"\n{'='*60}")
            print(f"Node: {node_name}")
            print(f"{'='*60}")
            
            if node_name == "clarify_with_user":
                if "messages" in node_output:
                    last_msg = node_output["messages"][-1]
                    print(f"\n{last_msg.content}")
            
            elif node_name == "write_research_brief":
                if "research_brief" in node_output:
                    print(f"\nResearch Brief Generated:")
                    print(f"{node_output['research_brief'][:500]}...")
            
            elif node_name == "supervisor":
                print(f"\nSupervisor planning research strategy...")
                if "supervisor_messages" in node_output:
                    last_msg = node_output["supervisor_messages"][-1]
                    if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                        print(f"Tool calls: {len(last_msg.tool_calls)}")
                        for tc in last_msg.tool_calls:
                            print(f"  - {tc['name']}")
            
            elif node_name == "supervisor_tools":
                print(f"\nExecuting supervisor's tool calls...")
                if "notes" in node_output:
                    print(f"Research notes collected: {len(node_output['notes'])}")
            
            elif node_name == "final_report_generation":
                if "final_report" in node_output:
                    print(f"\n" + "="*60)
                    print("FINAL REPORT GENERATED")
                    print("="*60 + "\n")
                    display(Markdown(node_output["final_report"]))
    
    print("\n" + "="*60)
    print("Research workflow completed!")
    print("="*60)

# Run the research
await run_research()

Starting research workflow...


Node: clarify_with_user

I have sufficient information to proceed with your sleep improvement research. I understand you're looking for evidence-based strategies to address your current sleep challenges: inconsistent bedtimes (10pm-1am), phone use in bed, and morning fatigue. I will now research the best scientifically-backed sleep hygiene practices and create a comprehensive, personalized sleep improvement plan tailored to your specific situation.

Node: write_research_brief

Research Brief Generated:
I want to improve my sleep quality and need a comprehensive, evidence-based sleep improvement plan. My current sleep challenges include: going to bed at inconsistent times (ranging from 10pm to 1am), using my phone in bed, and frequently feeling tired in the morning despite sleeping. Please research the most effective, scientifically-backed sleep hygiene strategies and interventions that specifically address irregular bedtime schedules, screen time/blue li

# Comprehensive Evidence-Based Sleep Improvement Plan

This plan addresses your specific sleep challenges—inconsistent bedtimes, phone use in bed, and morning fatigue—using scientifically-backed strategies from peer-reviewed sleep medicine research.

## Sleep Timing and Consistency Protocol

### The Critical Importance of Sleep Regularity

Research demonstrates that consistent sleep schedules are fundamental to health and performance. The National Sleep Foundation's 2023 consensus guidelines emphasize that maintaining the same bedtime and wake-up time within 30 minutes, including weekends, is associated with improved outcomes across multiple health dimensions including alertness, cardiovascular and metabolic health, inflammation, and mental health [1][2].

Your current 3-hour window of bedtime variability (10pm-1am) significantly impacts your circadian rhythms. A study of college students found that irregular sleepers showed delayed melatonon onset by 2.6 hours, later peak sleep propensity, and more daytime sleep episodes compared to regular sleepers. Despite sleeping the same total duration, irregular sleepers had lower academic performance, with each 10-point increase in sleep regularity associated with a 0.10 increase in GPA [3].

### Implementation Strategy for Consistent Sleep Timing

**Phase 1: Choose Your Target Schedule (Week 1)**
- Select a realistic bedtime between 10:30pm-11:00pm that allows for 7-9 hours of sleep
- Choose a consistent wake time that works with your schedule
- Use the gradual adjustment method: shift your current bedtime earlier by 15-20 minutes every 2-3 nights until you reach your target

**Phase 2: Strengthen Circadian Anchors (Weeks 2-4)**
- Get natural sunlight exposure within 30 minutes of waking to reset your internal clock
- Maintain your schedule within 30 minutes, even on weekends (limited "social jetlag")
- If you must have catch-up sleep, limit it to 1-2 additional hours on weekends, as recent research shows this can be beneficial for recovering from sleep debt [2]

**Expected Timeline**: Most people notice improvements within 1-2 weeks of consistent routines, with full benefits developing over several weeks [6].

## Screen Time and Blue Light Management

### The Science Behind Screen-Sleep Disruption

Recent large-scale studies provide compelling evidence about screen time's impact on sleep. Research of nearly 40,000 Norwegian university students found that each additional hour of screen time after going to bed increased insomnia symptoms by 59% and reduced sleep by an average of 24 minutes per night [9]. A parallel U.S. study showed that people using screens before bed had a 33% higher rate of poor sleep quality and slept approximately 50 minutes less per week [9].

Your phone use in bed is particularly problematic. A study of Saudi Arabian adults found that 95.1% had smartphones in bedrooms, with regular smartphone use associated with nearly double the risk of very poor sleep quality (OR 1.98) [10].

### Evidence-Based Screen Reduction Protocol

**The 10-3-2-1-0 Method**
This research-backed approach provides a structured wind-down routine [6]:
- **10 hours before bed**: No caffeine
- **3 hours before bed**: No alcohol or heavy meals  
- **2 hours before bed**: Stop work-related activities
- **1 hour before bed**: No screens (this is your key intervention point)
- **0**: No snooze button in the morning

**Specific Implementation for Phone Use**
1. **Create a charging station outside your bedroom**: This removes the temptation entirely
2. **Use an analog alarm clock**: Eliminate the "I need my phone for the alarm" excuse
3. **Implement the 20-minute rule**: If you can't sleep within 20-30 minutes, get up and go to another room until sleepy
4. **Evening phone shutdown ritual**: Set a specific time (1 hour before target bedtime) to put your phone in airplane mode

**Alternative Activities for Your Pre-Sleep Hour**
- Reading physical books or magazines
- Gentle stretching or yoga
- Meditation or breathing exercises
- Journaling
- Light household tasks

### Blue Light Considerations

While newer research suggests blue light may have less impact than previously believed, the behavioral and cognitive arousal from phone use remains problematic [7]. The issue isn't just the light—it's the mental stimulation, social media engagement, and the conditioned association between your bed and wakefulness.

## Morning Alertness and Fatigue Reduction

### Understanding Sleep Inertia

Your morning fatigue likely stems from sleep inertia—the grogginess, disorientation, and cognitive impairment immediately following waking. This typically lasts 15-60 minutes but can extend for hours, especially when awakening from deep (slow-wave) sleep [15].

Research shows sleep inertia can severely impair performance—sometimes worse than losing an entire night of sleep. The key insight is that sleep inertia is far more likely when someone is suddenly awakened from deep sleep phases [14].

### Evidence-Based Morning Alertness Protocol

**Immediate Post-Wake Strategies (0-20 minutes)**
1. **Light exposure**: Get bright light (ideally natural sunlight) within the first 30 minutes of waking. A randomized study found that light-emitting interventions significantly improved alertness and energy levels compared to control conditions [11]

2. **Strategic caffeine timing**: If you use caffeine, research shows it's most effective when consumed immediately upon waking rather than waiting. Consider caffeinated chewing gum for faster absorption than pills [13]

3. **Avoid critical decisions**: Wait at least 20 minutes before making important decisions or performing safety-critical activities like driving [12]

**Sleep Architecture Optimization**
- **Nap strategically**: Keep naps to 10-15 minutes or extend to ~90 minutes to avoid awakening from deep sleep
- **Avoid the snooze button**: Research shows repeated snooze button use can trigger sleep inertia episodes [14]
- **Time your sleep cycles**: Aim for sleep durations that align with 90-minute cycles (6, 7.5, or 9 hours) to increase chances of waking during lighter sleep phases

**Pre-Sleep Strategies to Improve Morning Alertness**
- Plan your sleep duration to avoid nighttime awakenings
- Ensure adequate prior sleep (avoid sleep debt accumulation)
- Consider a "coffee nap": consuming caffeine before a 20-minute nap can reduce post-nap grogginess

## Sleep Environment Optimization

### Temperature Control

The ideal bedroom temperature for optimal sleep is 65-68°F (18-20°C), with 65°F being the most commonly cited optimal temperature [4][1]. This cooler setting helps maintain the natural drop in core body temperature that occurs during sleep onset. Adults sleeping in optimized temperature environments report up to 73% better sleep quality according to National Sleep Foundation data [1].

### Light Management Protocol

Circadian rhythms are heavily influenced by light exposure patterns. Natural light during the day promotes cortisol production for alertness, while darkness triggers melatonin production for sleepiness [4].

**Evening Light Control**:
- Light levels above 10 lux in the evening can reduce slow-wave sleep and increase nighttime awakenings
- Install blackout curtains or room-darkening shades
- Cover LED lights on electronic devices
- Use dim, warm lighting (under 10 lux) for evening activities

**Morning Light Exposure**:
- Natural sunlight within 30 minutes of waking helps reset circadian rhythms
- If natural light isn't available, consider a light therapy device (2,000+ lux)

### Sound Optimization

Noise disruptions prevent deeper sleep stages, with sounds as low as 33 decibels affecting sleep quality. Loud noise disturbances can cause severe sleep fragmentation with negative impacts on physical and mental health [4].

**Sound Management Strategies**:
- Use earplugs, white noise machines, or fans for consistent background sound
- Consider pink noise, which a 2017 study found increased deep sleep and improved memory in older adults [1]
- Address partner-related noise issues (snoring, movement)

### Additional Environmental Factors

**Bedding and Mattress**: Newer mattresses can improve sleep quality and reduce back pain. Regular bedding maintenance (washing sheets at least every two weeks) reduces dust mites and promotes hygiene [4].

**Air Quality and Scents**: Some studies have found lavender essential oil can improve sleep quality and morning refreshment. Ensure good ventilation and consider air purification if needed [4].

## Additional Proven Sleep Interventions

### Relaxation and Wind-Down Techniques

**4-7-8 Breathing Technique**
This "relaxing breath" method involves:
- Inhaling for 4 counts
- Holding for 7 counts  
- Exhaling for 8 counts
- Repeating 4 times

A 2022 study showed this technique creates immediate heart rate variability changes and blood pressure reductions by shifting toward parasympathetic nervous system activation [6].

**Progressive Muscle Relaxation**
Reserve the hour before bed for winding down with activities like:
- Progressive muscle relaxation
- Gentle stretching
- Warm baths
- Light reading (physical books only)
- Deep breathing exercises

### Stimulus Control Protocol

Use your bed only for sleep and sex to create proper sleep associations. This is a core component of Cognitive Behavioral Therapy for Insomnia (CBT-I), the first-line treatment for chronic sleep problems [6].

**Implementation**:
- No work, eating, or entertainment in bed
- If unable to fall asleep within 20-30 minutes, get up and move to another room
- Return to bed only when feeling sleepy
- Maintain consistent wake times regardless of sleep quality the previous night

### Substance Management Guidelines

**Caffeine**: Research shows caffeine consumption after 3 PM significantly prohibits restful sleep by prolonging sleep onset time, limiting sleep depth, and increasing sleep disturbances [7].

**Alcohol**: While alcohol may initially cause drowsiness, it suppresses REM sleep during initial sleep cycles and leads to increased wake time and disturbances in later cycles [7].

**Timing Recommendations**:
- Last caffeine intake: 10 hours before bedtime
- Last alcohol consumption: 3 hours before bedtime
- Heavy meals: Complete 3 hours before bedtime

## Implementation Timeline and Expected Results

### Week 1: Foundation Setting
- Choose target bedtime and wake time
- Remove phone from bedroom
- Begin temperature optimization
- Start 4-7-8 breathing practice

### Weeks 2-3: Habit Reinforcement  
- Fine-tune sleep schedule using gradual adjustment
- Implement full 10-3-2-1-0 protocol
- Establish morning light exposure routine
- Practice stimulus control consistently

### Weeks 4-6: Optimization and Consistency
- Address any remaining environmental factors
- Solidify weekend schedule maintenance
- Evaluate and adjust based on sleep quality improvements
- Consider advanced interventions if needed

### Expected Timeline for Results

Research indicates that many people notice initial improvements within 1-2 weeks of implementing consistent sleep hygiene routines. A study of bedtime routine implementation found the most significant improvements occurred within the first three nights, with sleep onset latency decreasing by an average of 1.29 minutes per night during this period [12].

By the end of two weeks, participants experienced:
- Over 6 minutes reduction in time to fall asleep
- Over one hour increase in longest consolidated sleep period  
- Reduction in night wakings from over one per night to less than one per night

Full benefits typically develop over several weeks, especially when incorporating stimulus control and sleep restriction elements from CBT-I approaches.

### When to Seek Additional Help

If you don't see improvements after 4-6 weeks of consistent implementation, consider:
- Consultation with a sleep medicine specialist
- Evaluation for underlying sleep disorders
- Formal Cognitive Behavioral Therapy for Insomnia (CBT-I)
- Assessment of potential medical factors affecting sleep

This comprehensive plan addresses your specific challenges with evidence-based interventions. The key to success is consistency and patience, as your circadian system needs time to adjust to new patterns. Start with the foundational elements (consistent timing, phone removal, environment optimization) before adding advanced techniques.

### Sources

[1] Why Sleeping on a Consistent Schedule Is Important For Health: https://www.nytimes.com/2026/01/05/well/health-benefits-sleep-consistency.html

[2] Consistent Sleep Schedules with New Consensus Guideline: https://www.thensf.org/sleep-schedules-sleep-timing-guideline/

[3] Irregular sleep/wake patterns are associated with poorer academic performance: https://www.nature.com/articles/s41598-017-03171-4

[4] Bedroom Environment: What Elements Are Important?: https://www.sleepfoundation.org/bedroom-environment

[5] Sleep timing, sleep consistency, and health in adults: a systematic review: https://pubmed.ncbi.nlm.nih.gov/33054339/

[6] Sleep Hygiene Guide to Improve Your Sleep Quality: https://inkblotdoc.com/sleep-hygiene-that-actually-works-an-evidence-based-guide-now-with-4-7-8-breathing/

[7] Is screen time before bed bad for you? What new research suggests: https://www.advisory.com/daily-briefing/2024/06/17/blue-light

[8] Effects of evening smartphone use on sleep and declarative memory: https://academic.oup.com/braincomms/article/6/3/fcae173/7675955

[9] Screen time and sleep: What new studies reveal: https://sleepeducation.org/screen-time-and-sleep-what-new-studies-reveal/

[10] The impact of bedtime technology use on sleep quality and excessive daytime sleepiness: https://pmc.ncbi.nlm.nih.gov/articles/PMC8906383/

[11] An at-home evaluation of a light intervention to mitigate sleep inertia: https://www.sciencedirect.com/science/article/pii/S2352721823001651

[12] Implementation of a nightly bedtime routine: https://pmc.ncbi.nlm.nih.gov/articles/PMC6587179/

[13] Time to wake up: reactive countermeasures to sleep inertia: https://pmc.ncbi.nlm.nih.gov/articles/PMC5136610/

[14] Lifestyle changes can help fight sleep inertia: https://www.uclahealth.org/news/article/lifestyle-changes-can-help-fight-sleep-inertia

[15] Sleep Inertia: How to Combat Morning Grogginess: https://www.sleepfoundation.org/how-sleep-works/sleep-inertia


Research workflow completed!


## Task 9: Understanding the Output

Let's break down what happened:

### Phase 1: Clarification
The system checked if your request was clear. Since you provided specific details about your sleep issues, it likely proceeded without asking clarifying questions.

### Phase 2: Research Brief
Your request was transformed into a detailed research brief that guides the supervisor's delegation strategy.

### Phase 3: Supervisor Delegation
The supervisor analyzed the brief and decided how to break down the research:
- Used `think_tool` to plan strategy
- Called `ConductResearch` to delegate to researchers
- Each delegation specified a focused research topic (e.g., sleep hygiene, circadian rhythm, blue light effects)

### Phase 4: Parallel Research
Researchers worked on their assigned topics:
- Each researcher used web search tools to gather information
- Used `think_tool` to reflect after each search
- Decided when they had enough information
- Compressed their findings into clean summaries

### Phase 5: Final Report
All research findings were synthesized into a comprehensive sleep improvement plan with:
- Well-structured sections
- Evidence-based recommendations
- Practical action items
- Sources for further reading

## Task 10: Key Takeaways & Next Steps

### Architecture Benefits
1. **Dynamic Decomposition** - Research structure emerges from the question, not predefined
2. **Parallel Efficiency** - Multiple researchers work simultaneously
3. **ReAct Quality** - Strategic reflection improves search decisions
4. **Scalability** - Handles token limits gracefully through compression
5. **Flexibility** - Easy to add new tools and capabilities

### When to Use This Pattern
- **Complex research questions** that need multi-angle investigation
- **Comparison tasks** where parallel research on different topics is beneficial
- **Open-ended exploration** where structure should emerge dynamically
- **Time-sensitive research** where parallel execution speeds up results

### When to Use Section-Based Instead
- **Highly structured reports** with predefined format requirements
- **Template-based content** where sections are always the same
- **Sequential dependencies** where later sections depend on earlier ones
- **Budget constraints** where token efficiency is critical

### Extend the System
1. **Add MCP Tools** - Integrate specialized tools for your domain
2. **Custom Prompts** - Modify prompts for specific research types
3. **Different Models** - Try different Claude versions or mix models
4. **Persistence** - Use a real database for checkpointing instead of memory

### Learn More
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [Open Deep Research Repo](https://github.com/langchain-ai/open_deep_research)
- [Anthropic Claude Documentation](https://docs.anthropic.com/)
- [Tavily Search API](https://tavily.com/)

## ❓ Question #3:

What are the trade-offs of using parallel researchers vs. sequential research? When might you choose one approach over the other?

##### Answer:
The upside of parallel researchers is that they are faster, can work independently and are more flexible (emergent structure). The downside is increased cost and it may potentially introduce unnecessary complexity to tasks that don't require much decomposition.

Sequential research would be slower and less flexible. However, the output of each step would feed into each other subsequent step hence for some use cases it may be necessary (with clear constraints and dependencies).

I would use parallel for independent sub-tasks that need to be synthesisied at the end, e.g. scientific research.

I would use sequential for tasks that rely on each other (where each step depends on the previous step) for solving more linear problems, e.g. in conversational solutions where information needs to be gathered sequentially (customer data) and are subject to constraints (business logic).

## ❓ Question #4:

How would you adapt this deep research architecture for a production wellness application? What additional components would you need?

##### Answer: 
I would:
- consider the right model for the task weighing costs with capablities
- add guardrails in the generation prompt to ensure user safety and add a disclaimer
- store and retrieve where relevant user-specific data such as demographic details, health history, key lifestyle info, preferences, topics from previous interactions (long-term memory, a user profiledata store)
- add user authorisation for user data safety
- play around with the report structure to make sure the length and content are satisfactory for good user experience (the test report is quite lengthy)
- use logging for evaluation and optimization
- add additional features like daily check-in/ dairy




## 🏗️ Activity #2: Custom Wellness Research

Using what you've learned, run a custom wellness research task.

**Requirements:**
1. Create a wellness-related research question (exercise, nutrition, stress, etc.)
2. Modify the configuration for your use case
3. Run the research and analyze the output
4. Document what worked well and what could be improved

**Experiment ideas:**
- Research exercise routines for specific conditions (bad knee, lower back pain)
- Compare different stress management techniques
- Investigate nutrition strategies for specific goals
- Explore meditation and mindfulness research

**YOUR CODE HERE**

Back pain related request: benefits of strength training vs deep tissue massage

In [17]:
# Create your own wellness research request and run it
back_pain_research_request = """
# Compare the benefits of strength training with deep tissue massage for addressing upper back pain
"""

# Optionally modify the config
my_config = {
    "configurable": {
        "research_model": "anthropic:claude-sonnet-4-20250514",
        "research_model_max_tokens": 10000,
        "compression_model": "anthropic:claude-sonnet-4-20250514",
        "compression_model_max_tokens": 8192,
        "final_report_model": "anthropic:claude-sonnet-4-20250514",
        "final_report_model_max_tokens": 10000,
        "summarization_model": "anthropic:claude-sonnet-4-20250514",
        "summarization_model_max_tokens": 8192,
        "allow_clarification": True,
        "max_concurrent_research_units": 1,
        "max_researcher_iterations": 2,
        "max_react_tool_calls": 3,
        "search_api": "tavily",
        "max_content_length": 50000,
        "thread_id": str(uuid.uuid4())
    }
}

# Execute the graph
async def run_custom_research(request, config):
    """Run the research workflow and display results."""
    print("Starting research workflow...\n")
    
    async for event in graph.astream(
        {"messages": [{"role": "user", "content": request}]},
        config,
        stream_mode="updates"
    ):
        # Display each step
        for node_name, node_output in event.items():
            print(f"\n{'='*60}")
            print(f"Node: {node_name}")
            print(f"{'='*60}")
            
            if node_name == "clarify_with_user":
                if "messages" in node_output:
                    last_msg = node_output["messages"][-1]
                    print(f"\n{last_msg.content}")
            
            elif node_name == "write_research_brief":
                if "research_brief" in node_output:
                    print(f"\nResearch Brief Generated:")
                    print(f"{node_output['research_brief'][:500]}...")
            
            elif node_name == "supervisor":
                print(f"\nSupervisor planning research strategy...")
                if "supervisor_messages" in node_output:
                    last_msg = node_output["supervisor_messages"][-1]
                    if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                        print(f"Tool calls: {len(last_msg.tool_calls)}")
                        for tc in last_msg.tool_calls:
                            print(f"  - {tc['name']}")
            
            elif node_name == "supervisor_tools":
                print(f"\nExecuting supervisor's tool calls...")
                if "notes" in node_output:
                    print(f"Research notes collected: {len(node_output['notes'])}")
            
            elif node_name == "final_report_generation":
                if "final_report" in node_output:
                    print(f"\n" + "="*60)
                    print("FINAL REPORT GENERATED")
                    print("="*60 + "\n")
                    display(Markdown(node_output["final_report"]))
    
    print("\n" + "="*60)
    print("Research workflow completed!")
    print("="*60)

# Run the research
await run_custom_research(back_pain_research_request, my_config)


Starting research workflow...


Node: clarify_with_user

I have sufficient information to proceed with your request. I understand you want a comparison of the benefits of strength training versus deep tissue massage for addressing upper back pain. I will research and compare these two treatment approaches, examining their effectiveness, mechanisms of action, scientific evidence, and practical considerations for managing upper back pain. I'll now begin the research process.

Node: write_research_brief

Research Brief Generated:
I need a comprehensive comparison of the benefits of strength training versus deep tissue massage for addressing upper back pain. Please analyze both treatment approaches across multiple dimensions including: (1) effectiveness and clinical outcomes based on peer-reviewed scientific studies and randomized controlled trials, (2) mechanisms of action explaining how each treatment addresses upper back pain, (3) duration and frequency of treatment required for optimal

# Comprehensive Comparison: Strength Training vs. Deep Tissue Massage for Upper Back Pain

## Overview

Upper back pain affects millions of individuals worldwide, with treatment approaches ranging from active interventions like strength training to passive therapies such as deep tissue massage. While both approaches show promise in clinical settings, their effectiveness, mechanisms, and practical implementation differ significantly. This comprehensive analysis examines the evidence-based benefits and limitations of each treatment modality across multiple dimensions to guide clinical decision-making.

## Strength Training for Upper Back Pain

### Effectiveness and Clinical Outcomes

Strength training demonstrates robust clinical evidence for treating upper back and neck pain. A systematic review and network meta-analysis of 89 studies involving 5,578 patients found that resistance training was among the most effective exercise interventions for chronic back pain, achieving an 80% SUCRA score for both physical function improvement (pooled standardized mean difference of -1.14, 95% CI: -1.71 to -0.56) and mental health outcomes (mean difference of -1.26, 95% CI: -2.10 to -0.41) [1].

A randomized controlled trial with 20 sedentary male patients demonstrated that adding upper-extremity strengthening exercises to conventional programs produced superior outcomes. The supported exercise group showed significantly greater pain score reductions (3.8 points versus 2.7 points) and disability improvements (17.4 points versus 14.4 points decrease) compared to conventional exercise alone. Additionally, participants experienced a 34% increase in isometric extension strength [2].

Workplace-based research involving 449 office workers found that one hour of specific strength training weekly effectively reduced neck and shoulder pain. Among participants with baseline pain ≥3, neck pain reduction ranged from 1.14 to 1.88 points on a 0-9 scale, which researchers considered clinically significant. The study demonstrated that training three times weekly for 20 minutes showed the most consistent benefits [3].

### Mechanisms of Action

Strength training addresses upper back pain through multiple physiological pathways. The spinal muscular kinetic chain concept demonstrates that exercises applied to the upper spine contribute to pain relief throughout the back. In patients with chronic back pain, the average EMG frequency of the thoracic erector spinae muscle is significantly reduced compared to healthy individuals, and targeted strengthening can restore normal function [2].

The neurophysiological adaptations occur rapidly, with most initial strength improvements reflecting neural changes rather than muscle growth. During the first weeks of resistance training, the primary motor cortex shows reduced inhibition, decreased short-interval intracortical inhibition, and shorter cortical silent periods. These neural adaptations enhance motor unit recruitment and coordination, improving muscle function and reducing pain [4].

Muscle protein synthesis increases following resistance exercise, with elevated myofibrillar protein synthesis rates initially directed toward muscle repair and later supporting muscular hypertrophy. Eccentric exercise (muscle lengthening under tension) induces greater mechanical tension on muscle fibers, leading to more rapid addition of sarcomeres both in series and parallel, ultimately improving strength and endurance [5].

### Duration and Frequency Requirements

Research indicates flexible scheduling options for strength training implementation. The office worker study found that three distinct approaches were effective: training once weekly for 60 minutes, three times weekly for 20 minutes, or nine times weekly for 7 minutes over 20 weeks. The three-times-weekly approach showed optimal results with 60% adherence compared to 49% for once-weekly training [3].

A multicomponent intervention study demonstrated effectiveness with 12 weeks of strength training sessions lasting 45-60 minutes weekly, following 10 weeks of ergonomics training. Another successful protocol involved upper extremity strengthening performed three days per week for six weeks [2].

For program design, specific parameters depend on training goals:
- Strength development: >85% of 1RM, 2-6 sets, <6 reps, 2-5 minute rest
- Hypertrophy: 67-85% of 1RM, 3-6 sets, 6-12 reps, 30-90 second rest
- Muscular endurance: <67% of 1RM, 2-3 sets, >12 reps, <30 second rest [1]

### Cost-Effectiveness and Accessibility

Strength training demonstrates excellent cost-effectiveness with consistent positive return on investment ratios of 1.15:1 to 1.7:1 and cost savings ranging from $121-$2,834 through reduced healthcare utilization. Group-based supervised exercise therapy, while costing an additional $531 annually compared to home-based exercise, reduces direct medical costs by $122 per year [6].

Research suggests that gym-based exercise is more costly than home-based alternatives, with one supervised exercise intervention showing an incremental cost-effectiveness ratio of AU$64,235. However, exercise interventions are considered cost-effective compared to manual therapy or massage due to minimal adverse effects and sustained benefits [7].

### Safety Profile and Contraindications

Strength training has specific contraindications that must be considered. Individuals with previous spinal surgery, tumors, nerve root compression with neurological symptoms, spinal fractures, or infections may not be suitable candidates. Prolonged back pain can result in muscles with reduced mass, increased fatty content, and greater stiffness, potentially leading to easier fatigue and worsening pain initially [8].

Eccentric exercise may cause nonuniform muscle activation changes, potentially leading to strength imbalances. Initial effects include muscle microlesions, temporary pain, reduced fiber excitability, and short-term weakness. One study reported a 38% dropout rate, indicating potential challenges with program adherence and completion [9].

## Deep Tissue Massage for Upper Back Pain

### Research Limitations

Despite extensive searching, the available research specifically examining deep tissue massage for upper back pain is limited compared to strength training studies. Most massage research focuses on general massage therapy or combines multiple manual therapy techniques, making it difficult to isolate deep tissue massage effects specifically.

### Mechanisms and Clinical Context

While specific deep tissue massage studies are limited, research on thoracic manual therapy provides relevant insights. Manual therapy techniques work through neurophysiological mechanisms that produce short-term analgesia. A systematic review of thoracic manual therapy for subacromial pain syndrome found that when combined with thoracic extension exercises, results were more consistent and clinically meaningful than manual therapy alone [10].

The neurophysiological effects of spinal manipulation and manual therapy appear to be primarily short-term, which suggests that deep tissue massage may provide temporary relief but requires ongoing treatment for sustained benefits [11].

## Comparative Analysis

### Effectiveness and Evidence Quality

Strength training demonstrates superior evidence quality with multiple large-scale randomized controlled trials and systematic reviews. The evidence base includes specific protocols, dosing parameters, and long-term outcome data. In contrast, specific research on deep tissue massage for upper back pain is limited, with most studies focusing on broader manual therapy categories.

### Duration of Benefits

Strength training shows both immediate and long-term benefits. While initial neural adaptations occur within 3-4 weeks, sustained improvements in strength, endurance, and pain reduction continue with consistent training. Exercise programs can reduce the risk of new neck pain episodes by 53%, demonstrating significant preventive benefits [12].

Research on thoracic manipulation indicates that manual therapy effects may not persist beyond 2 months after discontinuation, suggesting that passive treatments like deep tissue massage may require ongoing sessions to maintain benefits [11].

### Cost Considerations

Strength training demonstrates superior long-term cost-effectiveness due to its preventive benefits and potential for self-directed implementation. Once proper technique is learned, individuals can continue strength training with minimal ongoing costs using basic equipment or bodyweight exercises.

Deep tissue massage typically requires professional administration, leading to higher ongoing costs without the same preventive benefits or self-management potential.

### Practical Implementation

Strength training offers greater flexibility in implementation, with evidence supporting various scheduling options (1×60, 3×20, or 9×7 minutes weekly). It can be performed in multiple settings including home, workplace, or gym environments. However, it requires initial instruction on proper technique and progressive overload principles.

Deep tissue massage requires trained practitioners and scheduled appointments, limiting accessibility and requiring ongoing financial investment.

### Suitability for Different Pain Causes

Strength training demonstrates broad applicability across various upper back pain causes:
- Occupational pain: Highly effective for work-related postural issues and repetitive strain
- Muscular weakness: Directly addresses underlying strength deficits
- Postural dysfunction: Improves muscle balance and spinal alignment
- Chronic pain conditions: Addresses both physical and neurophysiological aspects

Deep tissue massage may be more suitable for:
- Acute muscle tension and spasms
- Immediate pain relief needs
- Individuals unable to perform active exercise
- Complementary treatment alongside other interventions

## Recommendations for Combined Approaches

The evidence suggests that combining active and passive treatments may provide optimal outcomes. Research on thoracic manual therapy combined with exercise shows superior results compared to either approach alone [10]. A combined approach might include:

1. Initial deep tissue massage to address acute symptoms and muscle tension
2. Progressive introduction of strength training exercises as tolerance improves
3. Ongoing strength training for long-term management and prevention
4. Periodic massage sessions for acute flare-ups or maintenance

## Conclusion

Based on the available evidence, strength training demonstrates superior long-term effectiveness, cost-efficiency, and preventive benefits for upper back pain compared to deep tissue massage. The robust research base supporting strength training includes multiple high-quality randomized controlled trials showing significant improvements in pain, function, and quality of life.

While deep tissue massage may provide valuable short-term relief and could serve as a useful adjunct treatment, the limited specific research and temporary nature of benefits suggest it should not be the primary treatment approach. For optimal outcomes, individuals should prioritize strength training as the foundation of their treatment plan, potentially incorporating massage therapy for acute symptom management or as a complementary intervention.

The flexibility of strength training protocols, combined with its preventive benefits and potential for self-management, makes it the more practical and sustainable choice for most individuals with upper back pain. However, the specific cause, severity, and individual circumstances should guide treatment selection, with healthcare providers helping to determine the most appropriate approach for each patient.

### Sources

[1] Which specific modes of exercise training are most effective for treating non-specific chronic low back pain: https://bjsm.bmj.com/content/54/21/1279

[2] Effect of Upper-Extremity Strengthening Exercises on the Lumbar Strength and Disability in Patients with Chronic Low Back Pain: https://pmc.ncbi.nlm.nih.gov/articles/PMC5721192/

[3] Influence of frequency and duration of strength training for effective management of neck and shoulder pain: https://pmc.ncbi.nlm.nih.gov/articles/PMC3596862/

[4] Physiological and Neural Adaptations to Eccentric Exercise: https://pmc.ncbi.nlm.nih.gov/articles/PMC4620252/

[5] Mechanisms of resistance exercise‐induced muscle hypertrophy: https://pmc.ncbi.nlm.nih.gov/articles/PMC5157064/

[6] A Review of the Cost-Effectiveness of Supervised Exercise Therapy: https://www.scholars.northwestern.edu/en/publications/a-review-of-the-cost-effectiveness-of-supervised-exercise-therapy

[7] Summarizing the effects of different exercise types in chronic neck pain: https://pmc.ncbi.nlm.nih.gov/articles/PMC10568903/

[8] Weight Training Effectively Relieves Back Pain: https://www.precisionpaincarerehab.com/news-articles-pl409/blog/weight-training-effectively-relieves-back-pain-26065.html

[9] Multicomponent exercises to prevent and reduce back pain in elderly care nurses: https://link.springer.com/article/10.1186/s13102-022-00508-z

[10] Thoracic Manual Therapy With or Without Exercise Improves Pain and Disability in Subacromial Pain Syndrome: https://pmc.ncbi.nlm.nih.gov/articles/PMC12523727/

[11] The Effects of Spinal Manipulation Added to Exercise on Pain and Quality of Life in Patients with Thoracic Spinal Pain: https://pmc.ncbi.nlm.nih.gov/articles/PMC10159735/

[12] Effectiveness of Exercise Interventions for Preventing Neck Pain: https://www.jospt.org/doi/10.2519/jospt.2023.12063


Research workflow completed!


I run a back-pain related request to compare the benefits of strength training vs deep tissue massage for addressing back pain.

The quality of the report is really good, I'm quite happy with the result. I like the inline citations and the references at the end, with links that I can click and revise myself.

This is a typical research style output - which is not surprising of course since that's the design behind the current implementation. 

In a production grade application, however, I would consider, the following:
- The query was generic so the agent didn't ask for clarification. I would consider whether for my use case the system needs to ask the user clarifying questions, e.g. in a pure research context it doesn't, but in a wellness application, it should gather additional data.
- Depending on the use case and the target user, I would consider whether the language style needs adjusted, e.g. keep it very formal as is or make it easier to digest.
- In a prod research wellness app, I would store and fetch user-specific data to inform the search and the output.

